# ⭐ Lora Trainer by dmikey
**Version 2.0** - All-in-One Edition (February 2026) | `8e79c81`

This is based on the work of [Hollowstrawberry](https://github.com/hollowstrawberry/kohya-colab), [Kohya-ss](https://github.com/kohya-ss/sd-scripts) and [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb). Thank you!

### ⭕ Disclaimer
The purpose of this document is to research bleeding-edge technologies in the field of machine learning.  
Please read and follow the [Google Colab guidelines](https://research.google.com/colaboratory/faq.html) and its [Terms of Service](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|
|:--|:-:|
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/dmikey/kohya-colab/main/assets/github.svg)](https://github.com/dmikey/kohya-colab/blob/main/Lora_Trainer.ipynb) |

In [ ]:
#@title ## ⭐ LoRA Trainer - All-in-One
#@markdown ### ▶️ Setup
#@markdown Your project name will be the same as the folder containing your images. Spaces aren't allowed.
project_name = "my_lora" #@param {type:"string"}
#@markdown The folder structure doesn't matter and is purely for comfort.
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]
#@markdown Choose your training model:
training_model = "Anime (animefull-final-pruned-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "AnyLora (AnyLoRA_noVae_fp16-pruned.ckpt)", "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)"]
optional_custom_training_model_url = "" #@param {type:"string"}
custom_model_is_based_on_sd2 = False #@param {type:"boolean"}

#@markdown ### ▶️ Processing
resolution = 512 #@param {type:"slider", min:512, max:1024, step:128}
flip_aug = False #@param {type:"boolean"}
shuffle_tags = True #@param {type:"boolean"}
activation_tags = "1" #@param [0,1,2,3]

#@markdown ### ▶️ Steps
num_repeats = 10 #@param {type:"number"}
preferred_unit = "Epochs" #@param ["Epochs", "Steps"]
how_many = 10 #@param {type:"number"}
save_every_n_epochs = 1 #@param {type:"number"}
keep_only_last_n_epochs = 10 #@param {type:"number"}
train_batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}

#@markdown ### ▶️ Learning
unet_lr = 5e-4 #@param {type:"number"}
text_encoder_lr = 1e-4 #@param {type:"number"}
lr_scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler_number = 3 #@param {type:"number"}
lr_warmup_ratio = 0.05 #@param {type:"slider", min:0.0, max:0.5, step:0.01}
min_snr_gamma = True #@param {type:"boolean"}

#@markdown ### ▶️ Structure
lora_type = "LoRA" #@param ["LoRA", "LoCon"]
network_dim = 16 #@param {type:"slider", min:1, max:128, step:1}
network_alpha = 8 #@param {type:"slider", min:1, max:128, step:1}
conv_dim = 8 #@param {type:"slider", min:1, max:64, step:1}
conv_alpha = 4 #@param {type:"slider", min:1, max:64, step:1}

#@markdown ### ▶️ Optimizer
optimizer = "AdamW8bit" #@param ["AdamW8bit", "Prodigy", "DAdaptation", "AdamW", "Lion", "SGDNesterov"]

# =============================================================================
# STEP 1: Install Dependencies
# =============================================================================
import os
import sys
import subprocess
from time import time

print("=" * 80)
print("STEP 1: Installing Dependencies")
print("=" * 80)

# Detect environment
COLAB = 'google.colab' in str(get_ipython()) if 'get_ipython' in dir() else False
root_dir = "/content" if COLAB else os.path.expanduser("~/Loras")
repo_dir = os.path.join(root_dir, "kohya-trainer")

# Mount Google Drive if on Colab
if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

os.makedirs(root_dir, exist_ok=True)
os.chdir(root_dir)

print("\n🏭 Installing dependencies...\n")
t0 = time()

# Clone fresh kohya_ss
if os.path.exists(repo_dir):
    !rm -rf {repo_dir}

!git clone --quiet https://github.com/kohya-ss/sd-scripts {repo_dir}
os.chdir(repo_dir)

# Install system dependencies
!apt -y update -qq
!apt -y install aria2 -qq

# Install Python packages
print("Installing PyTorch and xformers...")
!pip install -q torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu121
!pip install -q xformers==0.0.23.post1 --index-url https://download.pytorch.org/whl/cu121

print("Installing kohya requirements...")
!pip install -q -r requirements.txt

print("Installing additional packages...")
!pip install -q prodigyopt lion-pytorch tensorboard

# Try to install compatible bitsandbytes
print("Installing bitsandbytes...")
bitsandbytes_available = False

# Detect actual CUDA version
import torch
cuda_version = torch.version.cuda
print(f"   Detected CUDA version: {cuda_version}")

# Install matching bitsandbytes version
if cuda_version and cuda_version.startswith("12"):
    !pip install -q bitsandbytes>=0.43.0
else:
    !pip install -q bitsandbytes==0.41.1

# Verify it actually works (import test)
try:
    import bitsandbytes as bnb
    # Force a real check - the import alone doesn't catch CUDA issues
    _ = bnb.optim.AdamW8bit
    print(f"✓ bitsandbytes {bnb.__version__} installed successfully")
    bitsandbytes_available = True
except Exception as e:
    print(f"⚠️ bitsandbytes not functional: {e}")
    print("   Uninstalling broken bitsandbytes to prevent crashes...")
    !pip uninstall -y bitsandbytes -q
    print("   Will use standard optimizers instead of 8-bit variants")

# Verify xformers installation
try:
    import xformers
    print(f"✓ xformers {xformers.__version__} installed successfully")
    xformers_available = True
except ImportError:
    print("⚠️ xformers not available - will disable in config")
    xformers_available = False

# Setup accelerate
from accelerate.utils import write_basic_config
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
os.makedirs(os.path.dirname(accelerate_config), exist_ok=True)
if not os.path.exists(accelerate_config):
    write_basic_config(save_location=accelerate_config)

# Environment variables
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["BITSANDBYTES_NOWELCOME"] = "1"
os.environ["SAFETENSORS_FAST_GPU"] = "1"

t1 = time()
print(f"\n✅ Installation finished in {int(t1-t0)} seconds.")

# =============================================================================
# STEP 2: Setup Training Configuration
# =============================================================================
import re
import toml

print("\n" + "=" * 80)
print("STEP 2: Setting Up Training Configuration")
print("=" * 80)

# Model URL mapping
if optional_custom_training_model_url:
    model_url = optional_custom_training_model_url
elif "AnyLora" in training_model:
    model_url = "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt"
elif "Anime" in training_model:
    model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"
else:
    model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors"

# Setup paths
if "/Loras" in folder_structure:
    main_dir = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
    log_folder = os.path.join(main_dir, "_logs")
    config_folder = os.path.join(main_dir, project_name)
    images_folder = os.path.join(main_dir, project_name, "dataset")
    output_folder = os.path.join(main_dir, project_name, "output")
else:
    main_dir = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
    images_folder = os.path.join(main_dir, "datasets", project_name)
    output_folder = os.path.join(main_dir, "output", project_name)
    config_folder = os.path.join(main_dir, "config", project_name)
    log_folder = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

# Create directories
for d in [main_dir, log_folder, images_folder, output_folder, config_folder]:
    os.makedirs(d, exist_ok=True)

# Validate dataset
print("\n💿 Checking dataset...")
project_name = project_name.strip()
if not project_name or any(c in project_name for c in " .()\"'\\/"):
    raise ValueError("Please choose a valid project name (no spaces or special characters).")

supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")
if not os.path.exists(images_folder):
    raise ValueError(f"Dataset folder doesn't exist: {images_folder}")

files = os.listdir(images_folder)
images = [f for f in files if f.lower().endswith(supported_types)]
if not images:
    raise ValueError(f"No images found in {images_folder}")

caption_extension = ".txt" if [f for f in files if f.lower().endswith(".txt")] else ""
keep_tokens = int(activation_tags)
shuffle_caption = shuffle_tags

# Calculate steps
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
steps_per_epoch = (len(images) * num_repeats) / train_batch_size
total_steps = max_train_steps or int(max_train_epochs * steps_per_epoch)
lr_warmup_steps = int(total_steps * lr_warmup_ratio)

print(f"📁 Found {len(images)} images with {num_repeats} repeats")
print(f"📈 {steps_per_epoch:.0f} steps per epoch, {total_steps} total steps")

# Download model
print("\n🔄 Downloading model...")
model_file = os.path.join(root_dir, model_url.split('/')[-1])

if not os.path.exists(model_file):
    !aria2c "{model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d "{root_dir}" -o "{os.path.basename(model_file)}"

if not os.path.exists(model_file):
    raise ValueError("Failed to download model")
print(f"✅ Model ready: {model_file}")

# Network settings
network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
    network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

# Optimizer settings for Prodigy/DAdaptation
optimizer_args = None
actual_unet_lr = unet_lr
actual_text_encoder_lr = text_encoder_lr
actual_lr_scheduler = lr_scheduler
actual_network_alpha = network_alpha
actual_optimizer = optimizer

# Check if bitsandbytes is available for 8-bit optimizers
if "8bit" in optimizer.lower() and not bitsandbytes_available:
    print(f"⚠️ {optimizer} requires bitsandbytes which is not available")
    print("   Falling back to regular AdamW optimizer")
    actual_optimizer = "AdamW"

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
    actual_unet_lr = 1.0
    actual_text_encoder_lr = 1.0
    actual_lr_scheduler = "constant_with_warmup"
    actual_network_alpha = network_dim
    optimizer_args = ["decouple=True", "weight_decay=0.01", "betas=[0.9,0.999]"]
    if optimizer.lower() == "prodigy":
        optimizer_args.extend(["d_coef=2", "use_bias_correction=True", "safeguard_warmup=True"])

lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
min_snr_gamma_value = 5.0 if min_snr_gamma else None

# Create config
print("\n📄 Creating config files...")
config_dict = {
    "additional_network_arguments": {
        "unet_lr": actual_unet_lr,
        "text_encoder_lr": actual_text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": actual_network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if actual_text_encoder_lr == 0 else None,
    },
    "optimizer_arguments": {
        "learning_rate": actual_unet_lr,
        "lr_scheduler": actual_lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if actual_lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if actual_lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if actual_lr_scheduler != "constant" else None,
        "optimizer_type": actual_optimizer,
        "optimizer_args": optimizer_args,
    },
    "training_arguments": {
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "train_batch_size": train_batch_size,
        "clip_skip": 2,
        "min_snr_gamma": min_snr_gamma_value,
        "seed": 42,
        "max_token_length": 225,
        "xformers": xformers_available,
        "lowram": COLAB,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": log_folder,
        "output_name": project_name,
        "log_prefix": project_name,
    },
    "model_arguments": {
        "pretrained_model_name_or_path": model_file,
        "v2": custom_model_is_based_on_sd2,
        "v_parameterization": True if custom_model_is_based_on_sd2 else None,
    },
    "saving_arguments": {
        "save_model_as": "safetensors",
    },
    "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
    },
    "dataset_arguments": {
        "cache_latents": True,
    },
}

# Remove None values
for key in config_dict:
    if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

with open(config_file, "w") as f:
    f.write(toml.dumps(config_dict))
print(f"📄 Config saved to {config_file}")

# Create dataset config
dataset_config_dict = {
    "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,
    },
    "datasets": [
        {
            "subsets": [
                {
                    "num_repeats": num_repeats,
                    "image_dir": images_folder,
                    "class_tokens": None if caption_extension else project_name
                }
            ]
        }
    ]
}

# Remove None values from general
dataset_config_dict["general"] = {k: v for k, v in dataset_config_dict["general"].items() if v is not None}
for subset in dataset_config_dict["datasets"][0]["subsets"]:
    for k in list(subset.keys()):
        if subset[k] is None:
            del subset[k]

with open(dataset_config_file, "w") as f:
    f.write(toml.dumps(dataset_config_dict))
print(f"📄 Dataset config saved to {dataset_config_file}")

# =============================================================================
# STEP 3: Start Training
# =============================================================================
print("\n" + "=" * 80)
print("STEP 3: Starting Training")
print("=" * 80 + "\n")

os.chdir(repo_dir)

# Set environment variables to suppress warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONWARNINGS'] = 'ignore'

cmd = [
    sys.executable, "-m", "accelerate.commands.launch",
    f"--config_file={accelerate_config_file}",
    "--num_cpu_threads_per_process=1",
    "train_network.py",
    f"--dataset_config={dataset_config_file}",
    f"--config_file={config_file}"
]

# Launch training with filtered output
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

skip_patterns = [
    "Unable to register cuDNN",
    "Unable to register cuBLAS",
    "computation placer already registered",
    "compiled without GPU support",
    "undefined symbol: cadam32bit",
    "SyntaxWarning:",
    "invalid escape sequence",
    '"is not" with \'str\' literal',
    "torchao.kernel.intmm",
    "non-existent",
    "CUDA_SETUP:",
    "CUDA SETUP:",
    "libcudart.so",
    "libbitsandbytes",
]

print("⭐ Training started - filtering warnings...\n")

last_step_line = ""
for line in process.stdout:
    stripped = line.strip()
    # Skip empty/trivial lines
    if stripped in ("", "False", "True"):
        continue
    # Skip known non-critical warnings
    if any(skip in line for skip in skip_patterns):
        continue
    # Deduplicate tqdm progress lines - only show the latest update per step
    if "steps:" in line and "it/s" in line or "s/it" in line:
        last_step_line = line
        continue
    # Flush the last progress line before printing a non-progress line
    if last_step_line:
        print(last_step_line, end='')
        last_step_line = ""
    print(line, end='')

# Print any remaining progress line
if last_step_line:
    print(last_step_line, end='')

process.wait()

print("\n" + "=" * 80)
if process.returncode == 0:
    print("✅ Training completed successfully!")
    print(f"📁 Your LoRA is saved in: {output_folder}")
else:
    print(f"⚠️ Training stopped with exit code {process.returncode}")
print("=" * 80)

## *️⃣ Extras

Optional utilities for dataset management.

In [ ]:
#@title ### 📂 Unzip dataset
#@markdown Upload a zip file and extract it to your dataset folder.
zip_file = "/content/drive/MyDrive/my_dataset.zip" #@param {type:"string"}
extract_to = "/content/drive/MyDrive/Loras/my_lora/dataset" #@param {type:"string"}

import os, zipfile

if 'google.colab' in str(get_ipython()) and not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_file, 'r') as f:
    f.extractall(extract_to)
print("✅ Done")

In [ ]:
#@title ### 🔢 Count files in folders
folder = "/content/drive/MyDrive/Loras" #@param {type:"string"}

import os

if 'google.colab' in str(get_ipython()) and not os.path.exists('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')

for root, dirs, files in os.walk(folder):
    dirs[:] = [d for d in dirs if d not in ('_logs', 'output')]
    images = len([f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    if images:
        print(f"📁 {root}: {images} images")

# 📈 TensorBoard
View training progress after running the trainer.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir={log_folder}/